In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Oanda')))

import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [2]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [16,64]
i_pair = instrument.Instrument.get_instrument_by_name("EUR_USD")

In [3]:
df = pd.read_pickle(utils.get_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [4]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [5]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2023-12-13T21:00:00.000000000Z,1.08814,1.08841,1.08734,1.08751,1.080515,1.078322
1,2023-12-13T22:00:00.000000000Z,1.08772,1.08834,1.08736,1.08786,1.081106,1.078492
2,2023-12-13T23:00:00.000000000Z,1.08786,1.08850,1.08759,1.08827,1.081714,1.078691
3,2023-12-14T00:00:00.000000000Z,1.08828,1.08953,1.08792,1.08932,1.082364,1.078891
4,2023-12-14T01:00:00.000000000Z,1.08933,1.08981,1.08856,1.08966,1.083046,1.079093


In [6]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return False

In [7]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [8]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
52,2023-12-18T01:00:00.000000000Z,1.09005,1.09008,1.08946,1.08976,1.091350,1.091535,-0.000185,0.000379,-1
89,2023-12-19T14:00:00.000000000Z,1.09620,1.09844,1.09592,1.09792,1.093654,1.093307,0.000348,-0.000025,1
130,2023-12-21T07:00:00.000000000Z,1.09451,1.09503,1.09424,1.09472,1.094874,1.095046,-0.000173,0.000046,-1
136,2023-12-21T13:00:00.000000000Z,1.09804,1.10026,1.09726,1.09911,1.095458,1.095451,0.000007,-0.000186,1
247,2023-12-29T04:00:00.000000000Z,1.10654,1.10688,1.10638,1.10668,1.107342,1.107549,-0.000207,0.000163,-1


In [9]:
df_trades['DELTA'] = ( df_trades.mid_c.diff() / i_pair.pipLocation ).shift(-1)
df_trades['GAIN'] = df_trades.DELTA * df_trades.IS_TRADE



In [10]:
df_plot = df_ma.iloc[0:500].copy()
df_plot.shape

(500, 10)

In [11]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()